# Библиотеки и функции

In [3]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab2


In [4]:
pip install vaderSentiment

In [5]:

from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pandas as pd
pd.options.display.float_format = '{:.3}'.format
from sklearn.feature_extraction.text import CountVectorizer
import datetime
from tqdm.notebook import tqdm
import sklearn
import math
import numpy as np
import scipy.stats as ss
import random
from collections import defaultdict
from sklearn.decomposition import PCA

import re
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
import textblob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob.sentiments import NaiveBayesAnalyzer
from gensim.models import Word2Vec

from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
analyzer = SentimentIntensityAnalyzer()
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('sentiwordnet')
stop_words=set(stopwords.words("english"))




from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 100% !important; } </style>'))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [6]:
def getSubjectivity(review):
    result = textblob.TextBlob(review, analyzer=NaiveBayesAnalyzer()).sentiment
    return (result.p_pos,result.p_neg)

def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs

    
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist
        
def sentiwordnetanalysis(pos_data):
    sentiment = 0
    tokens_count = 0
    for word, pos in pos_data:
        if not pos:
            continue
        lemma = lem.lemmatize(word, pos=pos)
        if not lemma:
            continue
        synsets = wordnet.synsets(lemma, pos=pos)
        if not synsets:
            continue
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        return swn_synset.pos_score(),swn_synset.neg_score()
    
def token_features(token, part_of_speech):
    if token.isdigit():
        yield "numeric"
    else:
        yield "token={}".format(token.lower())
        yield "token,pos={},{}".format(token, part_of_speech)
    if token[0].isupper():
        yield "uppercase_initial"
    if token.isupper():
        yield "all_uppercase"
    yield "pos={}".format(part_of_speech)

def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

# Преобразование данных

<center>данные Reddit</center>

author - автор поста - сообщество или персона

title - заголовок

score - количество лайков поста

id - id поста

subreddit - один из тэгов для парсинга (['Bitcoin','Crypto','Cryptocurrency','Cryptocurrencies','Blockchain','Binance'])

num_comments - общее число комментариев

body - содержание поста (только текст)

created - время создания

top 1-10 comment - содержание комментария

top 1-10 comment score - количество лайков комментария

top 1-10 comment author - автор комментария

top 1-10 comment time - время комментария

total comments score - сумма лайков комментариев под постом

subreddit subscribers - количетсво подписчиков (если пост оставлен пабликом)

<center>данные дневные и часовые</center>

time - время среза данных

high - максимальная цена

low - минимальная цена

open - цена открытия

volumefrom - минимальный объем сделки за период

volumeto - максимальный объем сделки за период

close - цена закрытия 

conversionType - техническое поле

conversionSymbol - техническое поле

top_tier_volume_quote - максимальный sell-side объем по BTC

top_tier_volume_base - максимальный buy-side объем по BTC

top_tier_volume_total - максимальный суммарный объем по BTC

cccagg_volume_quote - средневзешенный sell-side объем по BTC

cccagg_volume_base - средневзешенный buy-side объем по BTC

cccagg_volume_total - средневзешенный суммарный объем по BTC

total_volume_quote - суммарный sell-side объем по BTC

total_volume_base - суммарный buy-price объем по BTC

total_volume_total - суммарный объем по BTC

volume - суммарный объем биржи по всем валютам

<center>данные минутные</center>


Unix - время в формате Timestamp 

Date - обычный datetime формат

Symbol - BTC/USD - техническое поле

Open - Цена открытия

High - Максимальная цена

Low - Минимальная цена

Close - Цена закрытия

Volume BTC - volume_quote - sell-side объем по BTC

Volume USD - volume_base - суммарный buy-side объем по BTC

##  Чтение исходного массива

In [ ]:
reddit = pd.read_csv('reddit.csv',index_col=0)
reddit = reddit.rename(columns={'created': 'time'})
reddit['time'] = reddit['time'].astype('object')
print(reddit.shape)
reddit.head()

(3566, 50)


,author,title,score,id,subreddit,num_comments,body,time,top 1 comment,top 2 comment,top 3 comment,top 4 comment,top 5 comment,top 6 comment,top 7 comment,top 8 comment,top 9 comment,top 10 comment,top 1 comment score,top 2 comment score,top 3 comment score,top 4 comment score,top 5 comment score,top 6 comment score,top 7 comment score,top 8 comment score,top 9 comment score,top 10 comment score,top 1 comment author,top 2 comment author,top 3 comment author,top 4 comment author,top 5 comment author,top 6 comment author,top 7 comment author,top 8 comment author,top 9 comment author,top 10 comment author,top 1 comment time,top 2 comment time,top 3 comment time,top 4 comment time,top 5 comment time,top 6 comment time,top 7 comment time,top 8 comment time,top 9 comment time,top 10 comment time,total comments score,subreddit subscribers
0,Bitcoin_Magazine,Bitcoin Core 22.0 Released: What’s New,184,pnlhqv,Bitcoin,49,NaN,2021-09-13 21:36:21,tldr; Bitcoin Core 22.0 is the 22nd major rele...,> The Invisible Internet Project (I2P) is anot...,I don't think it would take that long enough now,Does it have i2p bakes in? Or do you need to r...,It is the best and will be better!,Anyone having trouble verifying bitcoin core 2...,Massive thanks to all the developers who worke...,Baby steps to greatness.,Bitcoin Core 22.0 release notes: https://bitco...,inb4 people complain about large version numbe...,22.0,16.0,5.0,4.0,3.0,3.0,2.0,2.0,1.0,1.0,coinfeeds-bot,mujshanan92,lucaslbtce,gingeropolous,stellamctrie,NeoLightwood,booleanlifeform,basic_user321,TheGreatMuffin,dlq84,2021-09-13 23:15:33,2021-09-13 22:02:02,2021-09-21 19:47:56,2021-09-14 02:51:13,2021-09-14 00:00:35,2021-09-14 09:19:25,2021-09-14 08:36:50,2021-09-13 22:35:15,2021-09-21 11:19:16,2021-09-14 10:35:25,58,3372694
0,rBitcoinMod,"Daily Discussion, September 21, 2021",89,psbyo9,Bitcoin,738,Please utilize this sticky thread for all gene...,2021-09-21 08:04:28,Public Service Announcement:\n\nhttps://bitinf...,China will bail out Everscam and US will annou...,Just can't stop panic-buying. It's like walkin...,Regardless of what happens I’m going to yolo t...,#Anyone was thinking at a price of BTC over 50...,Wake and take some Bitcoin off the exchanges,"""...omg I wish I bought that dip in September....",You guys are getting way too gloomy... where's...,96.8% of all #Bitcoin volume transacted on-cha...,It's BITCOIN's time of the month. Lots of liqu...,32.0,29.0,24.0,20.0,16.0,16.0,16.0,15.0,15.0,15.0,PeterShiffindisguise,Tummlerr,Romsel87,Huejazzcok,Tank1882,food_dudes_,Tummlerr,bogus83,hateschoolfml,Born_Cattle6575,2021-09-22 00:45:39,2021-09-21 14:30:09,2021-09-21 21:21:03,2021-09-22 03:13:37,2021-09-21 18:03:43,2021-09-21 08:22:00,2021-09-22 01:49:41,2021-09-21 18:18:00,2021-09-21 21:43:19,2021-09-21 22:34:39,822,3372694
0,alOOshXL,Me watching Bitcoin chart all day.,477,pssp4r,Bitcoin,29,NaN,2021-09-22 00:34:13,This is quite accurate...,"Same, but with less (muscle) gains.",That's Botchback,My man got the all dressed chips he’ll yeah,"Same, had a rain day and had to keep lowering ...",Why am i looking at my portfolio going down bu...,"Now we are watching you, watching bitcoin. If ...",You even lift bro,This dude's face looks like a deep fake,Pullbacks are a gift,21.0,12.0,7.0,5.0,3.0,3.0,3.0,3.0,2.0,2.0,MadM4X89,riscten,Nitemarex,skcib,Hexagonaria,ConsistentJacket2294,Cryptowake,SeatDangerous2391,Tank_Top_Terror,katkatstrat,2021-09-22 00:49:33,2021-09-22 02:39:35,2021-09-22 01:17:48,2021-09-22 03:49:17,2021-09-22 01:33:04,2021-09-22 01:56:04,2021-09-22 02:59:58,2021-09-22 03:48:28,2021-09-22 05:07:45,2021-09-22 05:04:47,72,3372694
0,Xaqaree,You Bitcoin fuckers do not understand money. I...,634,pso5rg,Bitcoin,414,I've been listening to Bitcoin podcasts and re...,2021-09-21 20:57:44,Way to put all of us in the same basket. I kno...,What if you don't like ham sandwiches,this is written by someone who does not unders...,Bitcoin doesn't care about how it should be us...,Someone had a lecture o

In [ ]:
new_reddit = []
for x in range(reddit.shape[0]):
    main_data = pd.DataFrame(reddit.iloc[x][['author','score','id','subreddit',
           'num_comments','body','time','total comments score', 'subreddit subscribers']]).T
    main_data['part'] = 'post'
    
    sub_data = pd.DataFrame([reddit.iloc[x,8:][y:-2][::10].reset_index(drop=True) for y in range(10)]).dropna(how='all').reset_index(drop=True)
    sub_data.columns = ['body','total comments score','author','created']
    sub_data['part'] = 'comment'
    
    new_reddit += [pd.concat([main_data,sub_data],axis=0).fillna(method='ffill')]
    
reddit = pd.concat(new_reddit).sort_values(by = 'time').reset_index(drop=True)
print(reddit.shape)
reddit.head()

(18405, 11)


,author,score,id,subreddit,num_comments,body,time,total comments score,subreddit subscribers,part,created
0,davidw_-,606,7jrba2,crypto,200,NaN,2017-12-14 15:38:42,3.16e+02,208884,post,NaN
1,andrewwalton,606,7jrba2,crypto,200,"""Crypto"" means secret. It's often short for ""c...",2017-12-14 15:38:42,5.0,208884,comment,2017-12-15 00:52:54
2,rubdos,606,7jrba2,crypto,200,"Sooooo, you planning to sell t-shirts that say...",2017-12-14 15:38:42,6.0,208884,comment,2018-01-20 11:09:09
3,mockfry,606,7jrba2,crypto,200,Thank you for voicing our collective frustration,2017-12-14 15:38:42,13.0,208884,comment,2017-12-14 16:55:37
4,airpod_sleeping_pod,606,7jrba2,crypto,200,"It is true, just crypto (word) became like a b...",2017-12-14 15:38:42,14.0,208884,comment,2017-12-14 17:09:08


## daily prices

### создание агрегированного массива

In [ ]:
data = pd.read_csv('BTC day.csv',index_col=0)[['time','close','total_volume_total']]
data['time'] = [datetime.datetime.fromtimestamp(x).date() for x in data['time']]
data['returns'] = data['close'].pct_change()
reddit_day = reddit.copy()
reddit_day['time'] = [datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date() for x in reddit_day['time']]


new_df = []
for uniq_time in np.unique(reddit_day['time']):
    current_time = reddit_day[reddit_day['time']==uniq_time]
    new_df += [[' '.join(current_time['author'].dropna()),sum(current_time['score'].dropna()),' '.join(np.unique(current_time['subreddit'].dropna())),\
    sum(np.unique(current_time['num_comments'].dropna())),' '.join(current_time['body'].dropna()),uniq_time,\
    sum(np.unique(current_time['total comments score'].dropna())),sum(np.unique(current_time['subreddit subscribers'].dropna()))]]
reddit_day = pd.DataFrame(new_df,columns=['author','score','subreddit','num_comments','body','time','total comments score','subreddit subscribers'])
print(reddit_day.shape)
reddit_day.head()

reddit_day = pd.merge(reddit_day, data, on="time").dropna(subset=['returns'])
numeric_day_data = reddit_day[['score','num_comments','total comments score','subreddit subscribers','total_volume_total','returns']]

(121, 8)


### анализ текста

In [ ]:
content = reddit_day['body'].tolist()
new_content = []
for text_slice in content:
    new_text_slice = re.sub('[^A-Za-z]+', ' ',' '.join([y.replace('\n',' ').replace('\\',' ').lower() for y in text_slice.split(' ') if '@' not in y and 'http' not in y and '#' not in y]))
    filtered_sent=[]
    for w in new_text_slice.split(' '):
        if w not in stop_words:
            filtered_sent.append(w)
    new_text_slice = ' '.join([lem.lemmatize(x) for x in filtered_sent])
    new_content += [new_text_slice]

reddit_day['content'] = new_content
reddit_day = reddit_day[[len(x.split())>1 for x in reddit_day['content']]].reset_index(drop=True)

reddit_day['bayes subjectivity'] = reddit_day['content'].apply(getSubjectivity)
reddit_day['neg subjectivity bayes'] = [x[1] for x in [y if y is not None else 0 for y in reddit_day['bayes subjectivity']]]
reddit_day['pos subjectivity bayes'] = [x[0] for x in [y if y is not None else 0 for y in reddit_day['bayes subjectivity']]]

reddit_day['Vader Sentiment'] = reddit_day['content'].apply(vadersentimentanalysis)
reddit_day['Vader neg'] = [x['neg'] for x in [y if y is not None else 0 for y in reddit_day['Vader Sentiment']]]
reddit_day['Vader neu'] = [x['neu'] for x in [y if y is not None else 0 for y in reddit_day['Vader Sentiment']]]
reddit_day['Vader pos'] = [x['pos'] for x in [y if y is not None else 0 for y in reddit_day['Vader Sentiment']]]

pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
reddit_day['POS tagged'] = reddit_day['content'].apply(token_stop_pos)
reddit_day['SWN Sentiment'] = reddit_day['POS tagged'].apply(sentiwordnetanalysis)
reddit_day['SWN pos'] = [x[0] for x in [y if y is not None else (0,0) for y in reddit_day['SWN Sentiment']]]
reddit_day['SWN neg'] = [x[1] for x in [y if y is not None else (0,0) for y in reddit_day['SWN Sentiment']]]

reddit_day = reddit_day.drop(labels=['POS tagged','SWN Sentiment','Vader Sentiment','bayes subjectivity'],axis=1)
reddit_day.to_csv('reddit daily with sentiment.csv')

### feature exctraction

In [ ]:
reddit_day = pd.read_csv('reddit daily with sentiment.csv',index_col=0)
reddit_day['content'] = [' '.join([y for y in x.split(' ') if len(y)>1]) for x in reddit_day['content']]
n_comp = min([math.floor(reddit_day.shape[0]/2),30])

In [ ]:
X_hashed = ((token_features(y, nltk.pos_tag(y.split(' ')))) for y in reddit_day['content'])
hasher = sklearn.feature_extraction.FeatureHasher(input_type='string')
X_hashed = hasher.transform(X_hashed).toarray()
scaled = sklearn.preprocessing.scale(X_hashed, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_hashed = pd.DataFrame(transformed.T[:n_comp]).T
scores_hashed.columns = [str(i) + ' hashed' for i in range(1,n_comp+1)]

vectorizer = CountVectorizer()
X_count_vector = vectorizer.fit_transform(reddit_day['content'].tolist()).toarray()

scaled = sklearn.preprocessing.scale(X_count_vector, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_count_vector = pd.DataFrame(transformed.T[:n_comp]).T
scores_count_vector.columns = [str(i) + ' count vector' for i in range(1,n_comp+1)]

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)
analyze = bigram_vectorizer.build_analyzer()
X_ngram_vector = bigram_vectorizer.fit_transform(reddit_day['content'].tolist()).toarray()

scaled = sklearn.preprocessing.scale(X_ngram_vector, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_ngram_vector = pd.DataFrame(transformed.T[:n_comp]).T
scores_ngram_vector.columns = [str(i) + ' ngram vector' for i in range(1,n_comp+1)]

reddit_day = pd.concat([reddit_day,scores_hashed,scores_count_vector,scores_ngram_vector],axis=1)
reddit_day.to_csv('reddit daily with PCA.csv')

### создание библиотеки фичей

In [ ]:
reddit_day = pd.read_csv('reddit daily with PCA.csv',index_col=0)

In [ ]:
X = reddit_day[[x for x in reddit_day.columns if x not in ['author','subreddit','body','total_volume_total','returns','time','content']]]
print('X:',X.shape)
Y = reddit_day[['total_volume_total','returns']]
print('Y:',Y.shape)

X: (116, 102)
Y: (116, 2)


In [ ]:
# создаем X^2 и перекрестные произведения формата X_3^n-r*X_2^n-i
new_column = []
new_column_names = []
for r in  tqdm(X.columns):
    for i in X.columns:
            new_column+=[X[r]*X[i]]
            new_column_names += ['{}*{}'.format(r,i)]

  0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:
new_features = pd.concat(new_column,axis=1)
new_features.columns = new_column_names
X_total = pd.concat([X,new_features],axis=1)
X_total.to_csv('reddit daily total X.csv')

### auto ML

In [ ]:
X_total = pd.read_csv('reddit daily total X.csv',index_col=0)

In [5]:
pip install xgboost

In [6]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
#from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_total, Y.iloc[:,[1]], test_size=0.3, random_state=123)
xg_reg = xgb.XGBRegressor()

In [ ]:
err = []
for x in tqdm(range(X_train.shape[1])): 
    xg_reg.fit(X_train.iloc[:,[x]],y_train)
    preds = xg_reg.predict(X_test.iloc[:,[x]])
    err += [np.sqrt(mean_absolute_percentage_error(y_test, preds))]

  0%|          | 0/10506 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:45] WARNING: /workspace/src/objectiv

In [ ]:
results_first_step = pd.DataFrame(err,index=X_train.columns).sort_values(by=0)
results_first_step.to_csv('xgboost daily auto ml mape.csv')
results_first_step.head()

,0
22 hashed*29 count vector,9.47
29 count vector*22 hashed,9.47
9 count vector*3 ngram vector,9.49
3 ngram vector*9 count vector,9.49
5 count vector*8 ngram vector,9.74


In [ ]:
instr = [results_first_step.index[0]]
for i in range(2,results_first_step.shape[0]):
    if variance_inflation_factor(X_train[instr+[results_first_step.index[i]]].values, len(instr))>5:
        pass
    else:
        instr +=[results_first_step.index[i]]

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
pd.DataFrame(instr).to_csv('instr daily.csv')

In [ ]:
xg_reg.fit(X_train[instr],y_train)
preds = xg_reg.predict(X_test[instr])
mean_absolute_percentage_error(y_test, preds)

[21:07:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


153.11389902777927

## hourly prices

In [ ]:
data = pd.read_csv('BTC hour.csv',index_col=0)[['time','close','total_volume_total']]
data['time'] = [datetime.datetime.fromtimestamp(x).replace(minute=0, second=0) for x in data['time']]
data['returns'] = data['close'].pct_change()
data['time'] = data['time'].astype('object')  


reddit_hour = reddit.copy()
reddit_hour['time'] = [datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(minute=0, second=0) for x in reddit_hour['time']]
reddit_hour['time'] = reddit_hour['time'].astype('object')  

new_df = []
for uniq_time in np.unique(reddit_hour['time']):
    current_time = reddit_hour[reddit_hour['time']==uniq_time]
    new_df += [[' '.join(current_time['author'].dropna()),sum(current_time['score'].dropna()),' '.join(np.unique(current_time['subreddit'].dropna())),\
    sum(np.unique(current_time['num_comments'].dropna())),' '.join(current_time['body'].dropna()),uniq_time,\
    sum(np.unique(current_time['total comments score'].dropna())),sum(np.unique(current_time['subreddit subscribers'].dropna()))]]
reddit_hour = pd.DataFrame(new_df,columns=['author','score','subreddit','num_comments','body','time','total comments score','subreddit subscribers'])
print(reddit_hour.shape)
reddit_hour.head()

reddit_hour['time'] = reddit_hour['time'].astype('object')  
reddit_hour = pd.merge(reddit_hour, data, on="time").dropna(subset=['returns'])
numeric_hour_data = reddit_hour[['score','num_comments','total comments score','subreddit subscribers','total_volume_total','returns']]

(952, 8)


In [ ]:
content = reddit_hour['body'].tolist()
new_content = []
for text_slice in content:
    new_text_slice = re.sub('[^A-Za-z]+', ' ',' '.join([y.replace('\n',' ').replace('\\',' ').lower() for y in text_slice.split(' ') if '@' not in y and 'http' not in y and '#' not in y]))
    filtered_sent=[]
    for w in new_text_slice.split(' '):
        if w not in stop_words:
            filtered_sent.append(w)
    new_text_slice = ' '.join([lem.lemmatize(x) for x in filtered_sent])
    new_content += [new_text_slice]

reddit_hour['content'] = new_content
reddit_hour = reddit_hour[[len(x.split())>1 for x in reddit_hour['content']]].reset_index(drop=True)

reddit_hour['bayes subjectivity'] = reddit_hour['content'].apply(getSubjectivity)
reddit_hour['neg subjectivity bayes'] = [x[1] for x in [y if y is not None else 0 for y in reddit_hour['bayes subjectivity']]]
reddit_hour['pos subjectivity bayes'] = [x[0] for x in [y if y is not None else 0 for y in reddit_hour['bayes subjectivity']]]

reddit_hour['Vader Sentiment'] = reddit_hour['content'].apply(vadersentimentanalysis)
reddit_hour['Vader neg'] = [x['neg'] for x in [y if y is not None else 0 for y in reddit_hour['Vader Sentiment']]]
reddit_hour['Vader neu'] = [x['neu'] for x in [y if y is not None else 0 for y in reddit_hour['Vader Sentiment']]]
reddit_hour['Vader pos'] = [x['pos'] for x in [y if y is not None else 0 for y in reddit_hour['Vader Sentiment']]]

pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
reddit_hour['POS tagged'] = reddit_hour['content'].apply(token_stop_pos)
reddit_hour['SWN Sentiment'] = reddit_hour['POS tagged'].apply(sentiwordnetanalysis)
reddit_hour['SWN pos'] = [x[0] for x in [y if y is not None else (0,0) for y in reddit_hour['SWN Sentiment']]]
reddit_hour['SWN neg'] = [x[1] for x in [y if y is not None else (0,0) for y in reddit_hour['SWN Sentiment']]]

reddit_hour = reddit_hour.drop(labels=['POS tagged','SWN Sentiment','Vader Sentiment','bayes subjectivity'],axis=1)
reddit_hour.to_csv('reddit hourly with sentiment.csv')

In [ ]:
reddit_hour = pd.read_csv('reddit hourly with sentiment.csv',index_col=0)
reddit_hour['content'] = [' '.join([y for y in x.split(' ') if len(y)>1]) for x in reddit_hour['content']]
n_comp = min([math.floor(reddit_hour.shape[0]/2),30])

In [ ]:
X_hashed = ((token_features(y, nltk.pos_tag(y.split(' ')))) for y in reddit_hour['content'])
hasher = sklearn.feature_extraction.FeatureHasher(input_type='string')
X_hashed = hasher.transform(X_hashed).toarray()
scaled = sklearn.preprocessing.scale(X_hashed, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_hashed = pd.DataFrame(transformed.T[:n_comp]).T
scores_hashed.columns = [str(i) + ' hashed' for i in range(1,n_comp+1)]

vectorizer = CountVectorizer()
X_count_vector = vectorizer.fit_transform(reddit_hour['content'].tolist()).toarray()

scaled = sklearn.preprocessing.scale(X_count_vector, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_count_vector = pd.DataFrame(transformed.T[:n_comp]).T
scores_count_vector.columns = [str(i) + ' count vector' for i in range(1,n_comp+1)]

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)
analyze = bigram_vectorizer.build_analyzer()
X_ngram_vector = bigram_vectorizer.fit_transform(reddit_hour['content'].tolist()).toarray()

scaled = sklearn.preprocessing.scale(X_ngram_vector, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_ngram_vector = pd.DataFrame(transformed.T[:n_comp]).T
scores_ngram_vector.columns = [str(i) + ' ngram vector' for i in range(1,n_comp+1)]

reddit_hour = pd.concat([reddit_hour,scores_hashed,scores_count_vector,scores_ngram_vector],axis=1)
reddit_hour.to_csv('reddit hourly with PCA.csv')

In [ ]:
reddit_hour = pd.read_csv('reddit hourly with PCA.csv' , index_col=0)

In [ ]:
X = reddit_hour[[x for x in reddit_hour.columns if x not in ['author','subreddit','body','total_volume_total','returns','time','content']]]
print('X:',X.shape)
Y = reddit_hour[['total_volume_total','returns']]
print('Y:',Y.shape)

X: (796, 102)
Y: (796, 2)


In [ ]:
# создаем X^2 и перекрестные произведения формата X_3^n-r*X_2^n-i*X_1^n-j
new_column = []
new_column_names = []
for r in  tqdm(X.columns):
    for i in X.columns:
            new_column+=[X[r]*X[i]]
            new_column_names += ['{}*{}'.format(r,i)]

new_features = pd.concat(new_column,axis=1)
new_features.columns = new_column_names
X_total = pd.concat([X,new_features],axis=1)
X_total.to_csv('reddit hourly total X.csv')

  0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:


X_part = X_total.iloc[:int(X_total.shape[0]*0.3)]
Y_part = Y.iloc[:int(Y.shape[0]*0.3)]

X_train, X_test, y_train, y_test = train_test_split(X_part, Y_part.iloc[:,[1]], test_size=0.3, random_state=123)
xg_reg = xgb.XGBRegressor()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
err = []
for x in tqdm(range(X_train.shape[1])): 
    xg_reg.fit(X_train.iloc[:,[x]],y_train)
    preds = xg_reg.predict(X_test.iloc[:,[x]])
    err += [np.sqrt(mean_absolute_percentage_error(y_test, preds))]

results_first_step = pd.DataFrame(err,index=X_train.columns).sort_values(by=0)
results_first_step.to_csv('xgboost hourly auto ml mape.csv')
results_first_step.head()

  0%|          | 0/10506 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:38:14] WARNING: /workspace/src/objectiv

,0
SWN neg*SWN pos,9.95
SWN pos*SWN neg,9.95
9 hashed*SWN neg,10.1
SWN neg*9 hashed,10.1
SWN neg*4 hashed,10.1


In [ ]:
instr = [results_first_step.index[0]]
for i in range(2,results_first_step.shape[0]):
    if variance_inflation_factor(X_train[instr+[results_first_step.index[i]]].values, len(instr))>5:
        pass
    else:
        instr +=[results_first_step.index[i]]
pd.DataFrame(instr).to_csv('instr hourly.csv')
xg_reg.fit(X_train[instr],y_train)
preds = xg_reg.predict(X_test[instr])
mean_absolute_percentage_error(y_test, preds)

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


[07:43:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


221.9054113744205

In [ ]:
pd.DataFrame(instr).to_csv('instr hourly.csv')

In [ ]:
### ВАЛИДАЦИЯ ПРОШЛА, НАДО ДИНАМИЧЕСКИ ГОНЯТЬ

## minutely prices

In [ ]:
data = pd.read_csv('BTC minute.csv',index_col=0).reset_index(drop=True)
data = data[['date','close','Volume BTC']]
data = data.rename(columns = {'date':'time', 'Volume BTC':'total_volume_total'})
data['returns'] = data['close'].pct_change()

data['time'] = [datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S") for x in data['time']]
data['time'] = data['time'].astype('object')  



reddit_minute = reddit.copy()
reddit_minute['time'] = [datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(second=0) for x in reddit_minute['time']]
reddit_minute['time'] = reddit_minute['time'].astype('object')  

new_df = []
for uniq_time in np.unique(reddit_minute['time']):
    current_time = reddit_minute[reddit_minute['time']==uniq_time]
    new_df += [[' '.join(current_time['author'].dropna()),sum(current_time['score'].dropna()),' '.join(np.unique(current_time['subreddit'].dropna())),\
    sum(np.unique(current_time['num_comments'].dropna())),' '.join(current_time['body'].dropna()),uniq_time,\
    sum(np.unique(current_time['total comments score'].dropna())),sum(np.unique(current_time['subreddit subscribers'].dropna()))]]
reddit_minute = pd.DataFrame(new_df,columns=['author','score','subreddit','num_comments','body','time','total comments score','subreddit subscribers'])
print(reddit_minute.shape)
reddit_minute.head()

reddit_minute['time'] = reddit_minute['time'].astype('object')  
reddit_minute = pd.merge(reddit_minute, data, on="time").dropna(subset=['returns'])
numeric_minute_data = reddit_minute[['score','num_comments','total comments score','subreddit subscribers','total_volume_total','returns']]

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(3220, 8)


In [ ]:
content = reddit_minute['body'].tolist()
new_content = []
for text_slice in content:
    new_text_slice = re.sub('[^A-Za-z]+', ' ',' '.join([y.replace('\n',' ').replace('\\',' ').lower() for y in text_slice.split(' ') if '@' not in y and 'http' not in y and '#' not in y]))
    filtered_sent=[]
    for w in new_text_slice.split(' '):
        if w not in stop_words:
            filtered_sent.append(w)
    new_text_slice = ' '.join([lem.lemmatize(x) for x in filtered_sent])
    new_content += [new_text_slice]

reddit_minute['content'] = new_content
reddit_minute = reddit_minute[[len(x.split())>1 for x in reddit_minute['content']]].reset_index(drop=True)

reddit_minute['bayes subjectivity'] = reddit_minute['content'].apply(getSubjectivity)
reddit_minute['neg subjectivity bayes'] = [x[1] for x in [y if y is not None else 0 for y in reddit_minute['bayes subjectivity']]]
reddit_minute['pos subjectivity bayes'] = [x[0] for x in [y if y is not None else 0 for y in reddit_minute['bayes subjectivity']]]

reddit_minute['Vader Sentiment'] = reddit_minute['content'].apply(vadersentimentanalysis)
reddit_minute['Vader neg'] = [x['neg'] for x in [y if y is not None else 0 for y in reddit_minute['Vader Sentiment']]]
reddit_minute['Vader neu'] = [x['neu'] for x in [y if y is not None else 0 for y in reddit_minute['Vader Sentiment']]]
reddit_minute['Vader pos'] = [x['pos'] for x in [y if y is not None else 0 for y in reddit_minute['Vader Sentiment']]]

pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
reddit_minute['POS tagged'] = reddit_minute['content'].apply(token_stop_pos)
reddit_minute['SWN Sentiment'] = reddit_minute['POS tagged'].apply(sentiwordnetanalysis)
reddit_minute['SWN pos'] = [x[0] for x in [y if y is not None else (0,0) for y in reddit_minute['SWN Sentiment']]]
reddit_minute['SWN neg'] = [x[1] for x in [y if y is not None else (0,0) for y in reddit_minute['SWN Sentiment']]]

reddit_minute = reddit_minute.drop(labels=['POS tagged','SWN Sentiment','Vader Sentiment','bayes subjectivity'],axis=1)
reddit_minute.to_csv('reddit minutely with sentiment.csv')

In [7]:
reddit_minute = pd.read_csv('reddit minutely with sentiment.csv',index_col=0)
reddit_minute['content'] = [' '.join([y for y in x.split(' ') if len(y)>1]) for x in reddit_minute['content']]
n_comp = min([math.floor(reddit_minute.shape[0]/2),30])

27

In [ ]:
for i in range(reddit_minute.shape[0]//100):

    X_hashed = ((token_features(y, nltk.pos_tag(y.split(' ')))) for y in reddit_local['content'])
    hasher = sklearn.feature_extraction.FeatureHasher(input_type='string')
    X_hashed = hasher.transform(X_hashed).toarray()
    scaled = sklearn.preprocessing.scale(X_hashed, with_mean=True, with_std=True)
    model = sklearn.decomposition.PCA(n_components = n_comp)
    model.fit(scaled)
    transformed = model.transform(scaled)
    scores_hashed = pd.DataFrame(transformed.T[:n_comp]).T
    scores_hashed.columns = [str(i) + ' hashed' for i in range(1,n_comp+1)]

    vectorizer = CountVectorizer()
    X_count_vector = vectorizer.fit_transform(reddit_local['content'].tolist()).toarray()

    scaled = sklearn.preprocessing.scale(X_count_vector, with_mean=True, with_std=True)
    model = sklearn.decomposition.PCA(n_components = n_comp)
    model.fit(scaled)
    transformed = model.transform(scaled)
    scores_count_vector = pd.DataFrame(transformed.T[:n_comp]).T
    scores_count_vector.columns = [str(i) + ' count vector' for i in range(1,n_comp+1)]

    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)
    analyze = bigram_vectorizer.build_analyzer()
    X_ngram_vector = bigram_vectorizer.fit_transform(reddit_local['content'].tolist()).toarray()

    scaled = sklearn.preprocessing.scale(X_ngram_vector, with_mean=True, with_std=True)
    model = sklearn.decomposition.PCA(n_components = n_comp)
    model.fit(scaled)
    transformed = model.transform(scaled)
    scores_ngram_vector = pd.DataFrame(transformed.T[:n_comp]).T
    scores_ngram_vector.columns = [str(i) + ' ngram vector' for i in range(1,n_comp+1)]

    reddit_local = pd.concat([reddit_local,scores_hashed,scores_count_vector,scores_ngram_vector],axis=1)
    reddit_local.to_csv( srt(i) + 'reddit minutely with PCA.csv')

In [ ]:
X = reddit_minute[[x for x in reddit_minute.columns if x not in ['author','subreddit','body','total_volume_total','returns','time','content']]]
print('X:',X.shape)
Y = reddit_minute[['total_volume_total','returns']]
print('Y:',Y.shape)

In [ ]:
# создаем X^2 и перекрестные произведения формата X_3^n-r*X_2^n-i
new_column = []
new_column_names = []
for r in  tqdm(X.columns):
    for i in X.columns:
            new_column+=[X[r]*X[i]]
            new_column_names += ['{}*{}'.format(r,i)]

new_features = pd.concat(new_column,axis=1)
new_features.columns = new_column_names
X_total = pd.concat([X,new_features],axis=1)
X_total.to_csv('reddit minutely total X.csv')

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_part = X_total.iloc[:int(X_total.shape[0]*0.3)]
Y_part = Y.iloc[:int(Y.shape[0]*0.3)]

X_train, X_test, y_train, y_test = train_test_split(X_part, Y_part.iloc[:,[1]], test_size=0.3, random_state=123)
xg_reg = xgb.XGBRegressor()

In [ ]:
err = []
for x in tqdm(range(X_train.shape[1])): 
    xg_reg.fit(X_train.iloc[:,[x]],y_train)
    preds = xg_reg.predict(X_test.iloc[:,[x]])
    err += [np.sqrt(mean_absolute_percentage_error(y_test, preds))]

results_first_step = pd.DataFrame(err,index=X_train.columns).sort_values(by=0)
results_first_step.to_csv('xgboost minutely auto ml mape.csv')
results_first_step.head()

In [ ]:
instr = [results_first_step.index[0]]
for i in range(2,results_first_step.shape[0]):
    if variance_inflation_factor(X_train[instr+[results_first_step.index[i]]].values, len(instr))>5:
        pass
    else:
        instr +=[results_first_step.index[i]]
pd.DataFrame(instr).to_csv('instr minutely.csv')
xg_reg.fit(X_train[instr],y_train)
preds = xg_reg.predict(X_test[instr])
mean_absolute_percentage_error(y_test, preds)

In [7]:
#@title Twitter
# Twitter

In [7]:
twitter = pd.read_csv('twitter.csv',index_col=0)
twitter['date'] = [datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S%z").replace(second=0,tzinfo=None) for x in twitter['date']]
twitter = twitter.rename(columns={'date':'time'})
twitter['time'] = twitter['time'].astype('object') 

twitter.head()

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,time,id,content,username,subreddit
0,2021-09-23 16:02:00,1.44e+18,@hardmoneymag Peter Schiff… once you hold gold...,BankerWeimar,Bitcoin
1,2021-09-23 16:02:00,1.44e+18,"They meant P$100,000 (2k USD) . \n#Bitcoin #Bi...",bong_gonggong,Bitcoin
2,2021-09-23 16:02:00,1.44e+18,#Bitcoin $BTC https://t.co/rei3HRtFc0 https://...,DollaBillCrypto,Bitcoin
3,2021-09-23 16:02:00,1.44e+18,Bitcoin - $43930.0 (€37438.0)\nEthereum - ...,CryptoBotPrice,Bitcoin
4,2021-09-23 16:02:00,1.44e+18,Satoshi Spekülasyonu: 2012’den Beri Uykuda Ola...,turkcekripto,Bitcoin


In [8]:
data = pd.read_csv('BTC minute.csv',index_col=0).reset_index(drop=True)
data = data[['date','close','Volume USD']]
data = data.rename(columns = {'date':'time', 'Volume USD':'total_volume_total'})
data['returns'] = data['close'].pct_change()
data['time'] = [datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S") for x in data['time']]
data['time'] = data['time'].astype('object') 

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [13]:
new_df = []
for uniq_time in tqdm(np.unique(twitter['time'])):
    current_time = twitter[twitter['time']==uniq_time]
    new_df += [[uniq_time,' '.join(current_time['content'].dropna())]]

  0%|          | 0/71484 [00:00<?, ?it/s]

In [21]:
twitter = pd.DataFrame(new_df,columns = ['time','content'])
twitter['time'] = twitter['time'].astype('object') 
twitter = pd.merge(twitter, data, on="time").dropna(subset=['returns'])
twitter = twitter[['time','content','total_volume_total','returns']]

In [27]:
content = twitter['content'].tolist()
new_content = []
for text_slice in content:
    new_text_slice = re.sub('[^A-Za-z]+', ' ',' '.join([y.replace('\n',' ').replace('\\',' ').lower() for y in text_slice.split(' ') if '@' not in y and 'http' not in y and '#' not in y]))
    filtered_sent=[]
    for w in new_text_slice.split(' '):
        if w not in stop_words:
            filtered_sent.append(w)
    new_text_slice = ' '.join([lem.lemmatize(x) for x in filtered_sent])
    new_content += [new_text_slice]

twitter['content'] = new_content
twitter = twitter[[len(x.split())>1 for x in twitter['content']]].reset_index(drop=True)

In [7]:
twitter = pd.read_csv('twitter main.csv',index_col=0)
twitter['content'] = [' '.join([y for y in x.split(' ') if len(y)>1]) for x in twitter['content']]
n_comp = min([math.floor(twitter.shape[0]/2),30])

In [ ]:
X_hashed = ((token_features(y, nltk.pos_tag(y.split(' ')))) for y in twitter['content'])
hasher = sklearn.feature_extraction.FeatureHasher(input_type='string')
X_hashed = hasher.transform(X_hashed).toarray()
scaled = sklearn.preprocessing.scale(X_hashed, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_hashed = pd.DataFrame(transformed.T[:n_comp]).T
scores_hashed.columns = [str(i) + ' hashed' for i in range(1,n_comp+1)]

vectorizer = CountVectorizer()
X_count_vector = vectorizer.fit_transform(twitter['content'].tolist()).toarray()

scaled = sklearn.preprocessing.scale(X_count_vector, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_count_vector = pd.DataFrame(transformed.T[:n_comp]).T
scores_count_vector.columns = [str(i) + ' count vector' for i in range(1,n_comp+1)]

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)
analyze = bigram_vectorizer.build_analyzer()
X_ngram_vector = bigram_vectorizer.fit_transform(twitter['content'].tolist()).toarray()

scaled = sklearn.preprocessing.scale(X_ngram_vector, with_mean=True, with_std=True)
model = sklearn.decomposition.PCA(n_components = n_comp)
model.fit(scaled)
transformed = model.transform(scaled)
scores_ngram_vector = pd.DataFrame(transformed.T[:n_comp]).T
scores_ngram_vector.columns = [str(i) + ' ngram vector' for i in range(1,n_comp+1)]

twitter = pd.concat([twitter,scores_hashed,scores_count_vector,scores_ngram_vector],axis=1)
twitter.to_csv('twitter with PCA.csv')

In [ ]:
X = twitter[[x for x in twitter.columns if x not in ['author','subreddit','body','total_volume_total','returns','time','content']]]
print('X:',X.shape)
Y = twitter[['total_volume_total','returns']]
print('Y:',Y.shape)

In [ ]:
# создаем X^2 и перекрестные произведения формата X_3^n-r*X_2^n-i
new_column = []
new_column_names = []
for r in  tqdm(X.columns):
    for i in X.columns:
            new_column+=[X[r]*X[i]]
            new_column_names += ['{}*{}'.format(r,i)]

new_features = pd.concat(new_column,axis=1)
new_features.columns = new_column_names
X_total = pd.concat([X,new_features],axis=1)
X_total.to_csv('reddit total X.csv')